In [2]:
import torch
import sys

NUM_DATA = 500
t = torch.rand(NUM_DATA, dtype=torch.float32)
t.element_size() * t.numel()

2000

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T
import os.path as osp

path = osp.join('/data/gangda', 'ogb')
dataset = PygNodePropPredDataset(name='ogbn-products', root=path, transform=T.ToSparseTensor())
data = dataset[0]

In [20]:
data.adj_t.set_value_(torch.rand(data.num_edges), layout='csc')

SparseTensor(row=tensor([      0,       0,       0,  ..., 2449028, 2449028, 2449028]),
             col=tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
             val=tensor([0.3932, 0.8904, 0.9028,  ..., 0.4292, 0.5621, 0.6839]),
             size=(2449029, 2449029), nnz=123718280, density=0.00%)

In [22]:
from torch_geometric_autoscale import metis, permute, SubgraphLoader

perm, ptr = metis(data.adj_t, 4, log=True)
data_ = permute(data, perm, log=True)

Computing METIS partitioning with 4 parts... Done! [14.74s]
Permuting data... Done! [29.68s]


In [125]:
degree = data_.adj_t.sum(dim=1)
indptr, indices, value = data_.adj_t.csr()

In [134]:
i = 0
print(degree[i].item())
value[indptr[i]:indptr[i+1]].sum().item()

2.894808769226074


2.8948090076446533

In [75]:
data_list = list(SubgraphLoader(data_, ptr, batch_size=1, shuffle=False))
sub_data = data_list[0]

Pre-processing subgraphs... Done! [3.67s]


In [84]:
res = sub_data.data.adj_t.csr()
res[2][:6].sum()

tensor(2.8948)

In [86]:
adj = data.adj_t.t()
perm, ptr = metis(adj, 4, log=True)
data_ = permute(data, perm, log=True)

SparseTensor(row=tensor([      0,       0,       0,  ..., 2449028, 2449028, 2449028]),
             col=tensor([    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]),
             val=tensor([0.7309, 0.3331, 0.4298,  ..., 0.7652, 0.4662, 0.5261]),
             size=(2449029, 2449029), nnz=123718280, density=0.00%)

In [85]:
data_.adj_t.sum(dim=1).to(torch.float)

tensor([ 2.8948,  8.1311,  5.3858,  ...,  4.2185, 61.1699, 19.1762])

In [ ]:
d = {1: (1, 1),
     2: (2, 2),
     3: (3, 3)}
a = list(d.values())
a[0] = (2, 2)
for i, j in d.values():
     print(i, j)

In [8]:
1.0435e+02

104.35

In [2]:
import dgl
import torch
from ogb.nodeproppred import DglNodePropPredDataset
from scipy.sparse import csr_matrix

dataset = DglNodePropPredDataset(name='ogbn-products', root='/data/gangda/dgl')
g, labels = dataset[0]

edge_weight = torch.rand(g.num_edges())
g_csr_format = csr_matrix((edge_weight, (g.edges()[0], g.edges()[1])), shape=(g.num_nodes(), g.num_nodes()))

In [3]:
from tqdm import tqdm

weighted_degrees = []

for i in tqdm(range(len(g_csr_format.indptr) - 1)):
    start = g_csr_format.indptr[i]
    end = g_csr_format.indptr[i + 1]

    weighted = 0
    for j in range(start, end):
        u = g_csr_format.indices[j]
        weighted += g_csr_format.data[u]

    weighted_degrees.append(weighted)

weighted_degrees = torch.tensor(weighted_degrees)

100%|██████████| 2449029/2449029 [00:46<00:00, 52173.28it/s]


In [53]:
weighted_degrees = torch.zeros(g.num_nodes())

for i in tqdm(range(g.num_nodes())):
    start = g_csr_format.indptr[i]
    end = g_csr_format.indptr[i + 1]
    weighted_degrees[i] = edge_weight[start: end].sum()

weighted_degrees

100%|██████████| 2449029/2449029 [00:14<00:00, 169944.22it/s]


tensor([91.1288, 48.1758, 19.3833,  ..., 22.1408,  3.6230, 18.9324])

In [9]:
partitions = dgl.metis_partition(g, k=4, extra_cached_hops=1, reshuffle=False, balance_edges=False, mode='k-way')

Convert a graph into a bidirected graph: 4.650 seconds, peak memory: 39.748 GB
Construct multi-constraint weights: 0.001 seconds, peak memory: 39.748 GB


[15:20:13] /opt/dgl/src/graph/transform/metis_partition_hetero.cc:89: Partition a graph with 2449029 nodes and 123718408 edges into 4 parts and get 3130186 edge cuts


Metis partitioning: 21.856 seconds, peak memory: 39.748 GB
Split the graph: 15.728 seconds
Construct subgraphs: 2.534 seconds


In [66]:
partitions.adj_

Graph(num_nodes=896044, num_edges=32780964,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'inner_node': Scheme(shape=(), dtype=torch.int32), 'part_id': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'inner_edge': Scheme(shape=(), dtype=torch.int8)})

In [54]:
weighted_degrees_p0 = weighted_degrees[partitions[0].ndata['_ID']]
edge_weights_p0 = edge_weight[partitions[0].edata['_ID']]

In [55]:
partitions[0].edges()[0][partitions[0].edges()[1] == 292438]

tensor([383408, 125711, 479035])

In [56]:
weighted_degrees_p0[292438]

tensor(2.2358)

In [60]:
edge_weights_p0[partitions[0].edges()[1] == 292438].sum()

tensor(1.6434)